In [38]:
import numpy as np
import pandas as pd
import psycopg2
import cx_Oracle
import datetime
import os
%matplotlib inline
from IPython.display import display
from scipy import stats
from datetime import timedelta
from collections import Counter
from openpyxl import load_workbook

## Existing customers per site

In [2]:
query = """SELECT cop_c_id, cop_pref_o_ext_s_id, cop_g_order_count, cop_g_net_total, cop_g_cm2
FROM public.customer_order_profile
where cop_g_cm2 is not null
"""

In [3]:
conn = psycopg2.connect("dbname='savings_plan_db' user='dba' host='savings-plan-prod-c-savings-plan-db-readonly.ccsod6gcmvcs.eu-central-1.rds.amazonaws.com' password='m4d_l0y_db4?'")
curs = conn.cursor()
data = pd.read_sql(query, con = conn)
conn.close()

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9404913 entries, 0 to 9404912
Data columns (total 5 columns):
cop_c_id               int64
cop_pref_o_ext_s_id    int64
cop_g_order_count      int64
cop_g_net_total        float64
cop_g_cm2              float64
dtypes: float64(2), int64(3)
memory usage: 358.8 MB


In [5]:
data['CM2%'] = data['cop_g_cm2'] / data['cop_g_net_total'] * 100

In [6]:
data.sample(10)

,cop_c_id,cop_pref_o_ext_s_id,cop_g_order_count,cop_g_net_total,cop_g_cm2,CM2%
3823702,16006642,21,1,32.546567,8.430117,25.901709
4491846,16851440,1,2,127.311772,22.714525,17.841653
7208413,20566446,18,1,162.034800,14.864600,9.173708
4459969,16811520,107,1,70.239669,13.188219,18.776027
4178474,16456412,24,1,30.162510,8.963590,29.717653
5782906,18590355,1,1,116.487395,33.421395,28.690997
6701532,19840202,15,1,33.345455,-0.946421,-2.838231
4892125,17378792,12,6,158.372429,7.502094,4.736995
6067798,18976877,4,4,201.841666,26.572635,13.165089
2954900,14850547,16,2,105.404522,27.813054,26.386965


In [7]:
site_dict = {1: 'Deutschland',
 2: 'International',
 3: 'United Kingdom',
 4: 'France',
 7: 'Netherlands',
 8: 'Poland',
 11: 'Eire',
 12: 'Italy',
 14: 'Belgium',
 15: 'Spain',
 16: 'Czech',
 18: 'Finland',
 19: 'Slovakia',
 20: 'Russia',
 21: 'Denmark',
 22: 'Hungary',
 23: 'Slovenia',
 24: 'Romania',
 25: 'Switzerland',
 26: 'Sweden',
 28: 'Portugal',
 29: 'Croatia',
 30: 'Bulgaria',
 31: 'Norway',
 32: 'Greece'}

In [8]:
np.sort(data['cop_pref_o_ext_s_id'].unique())[:25]

array([ 1,  2,  3,  4,  7,  8, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 28, 29, 30, 31, 32], dtype=int64)

In [9]:
# Get active customers from Z+ in the last 3 years
data = data[data['cop_pref_o_ext_s_id'].isin(np.sort(data['cop_pref_o_ext_s_id'].unique())[:25])]

In [25]:
print('Z+ Active customers: ', len(data))

Z+ Active customers:  8193374


In [10]:
# Get all the existing customers having 2 or more orders finished
#data = data[data['cop_g_order_count'] >= 2]

Let's get from customers_zooprime_plans every customer ID whose SP has taken place in the last three years. Following this method we will be able to separate from existing customers those considered as renewals while the others would need to meet the Tier 4 or 5 thresholds criteria.

In [10]:
query = """select distinct czp_k_id
from customers_zooprime_plans
where czp_start_date >= trunc(sysdate - (365 * 3))
and czp_cancel_u_id is NULL
"""

In [11]:
conn = cx_Oracle.connect('readonly', 'read_123', 'db-zpmstb-01:1521/zpmstb.web.zooplus.de', threaded=True, encoding = "UTF-8", nencoding = "UTF-8")
curs = conn.cursor()
o_data = pd.read_sql(query, con = conn)
conn.close()

In [13]:
sp_customers = o_data['CZP_K_ID'].values

In [26]:
len(sp_customers)

1648974

In [14]:
data['is_renewal'] = (data['cop_c_id'].isin(sp_customers)) * 1

In [16]:
data.sample(10)

,cop_c_id,cop_pref_o_ext_s_id,cop_g_order_count,cop_g_net_total,cop_g_cm2,CM2%,is_renewal
2569485,14336530,1,3,65.859813,14.956042,22.708904,0
7047369,20338082,12,2,60.213115,9.674452,16.067018,0
3083583,15020243,4,3,117.433333,1.123532,0.956740,0
8224741,2300149,1,1,18.495327,1.413327,7.641536,0
230809,9051285,25,6,1384.709113,235.616280,17.015579,1
8456622,3653066,4,1,41.633333,12.621058,30.314791,0
4549614,16927689,26,1,30.732154,4.339279,14.119671,0
1144031,12000242,4,1,49.800000,16.462000,33.056225,0
7631355,12095240,4,12,1093.166599,190.264644,17.404908,1
2034518,13515087,1,6,264.569811,56.444256,21.334352,1


Customers having the NC SP need to be considered as existing customers because their CM2% in the future will be evaluated with the renewal threshold. Therefore, from our data, we will get as 'existing customers' those having at least 1 SP in the last three years (is_renewal = 1) and also those with at least 2 orders finished (Tier 4 & 5 thresholds for accessing 1st SP).

In [28]:
data = data[(data['is_renewal'] == 1) | (data['cop_g_order_count'] >= 2)]

In [29]:
len(data)

4966487

In [30]:
# Looking at thresholds we have more than 1.6M customers evaluated through renewal thresholds (low, high)
# and around 3.3M using Tier 4 & 5 thresholds for their 1st SP 
data['is_renewal'].value_counts()

0    3321473
1    1645014
Name: is_renewal, dtype: int64

In [66]:
file = 'New_routine_parameters.xlsx'
out_path = os.getcwd() + '\\' + file
writer = pd.ExcelWriter(out_path, engine='xlsxwriter')
workbook = writer.book
format1 = workbook.add_format({'num_format': '0.00'})

In [67]:
out_path

'C:\\Users\\albertoma\\Desktop\\Savings_plan\\postgres_conn\\New_routine_parameters.xlsx'

In [68]:
for i in np.sort(data['cop_pref_o_ext_s_id'].unique())[:25]:
    cus_data = data[data['cop_pref_o_ext_s_id'] == i]
    cus_amount = cus_data['cop_c_id'].count()
    cus_renewal = cus_data[cus_data['is_renewal'] == 1]
    cus_first_sp = cus_data[cus_data['is_renewal'] == 0]
    print('SITE ' + str(i) + ' [' + site_dict[i] + ']')
    print('')
    print('Total amount of existing customers: ', cus_amount)
    print('Renewals: ', cus_renewal['cop_c_id'].count())
    print('First SP: ', cus_first_sp['cop_c_id'].count())
    print('')
    print('Renewals Table')
    df_renewals = pd.DataFrame(columns=['THRESHOLD_%', 'ELIG_CUS', '%_RENEW', '%_TOTAL'])
    for j in ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15',
             '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']:
        cus_elig_renewal = cus_renewal[cus_renewal['CM2%'] >= float(j)]['cop_c_id'].count()
        elig_perc_r = cus_elig_renewal / cus_renewal['cop_c_id'].count() * 100
        elig_perc_r_from_total = cus_elig_renewal / cus_amount * 100
        lr = list([float(j), cus_elig_renewal, elig_perc_r, elig_perc_r_from_total])
        df1 = pd.DataFrame([lr], columns=df_renewals.columns)
        df_renewals = df_renewals.append(df1, ignore_index=True)
    display(df_renewals.round(2))
    print('')
    print('First SP Table')
    df_first_sp = pd.DataFrame(columns=['THRESHOLD_%', 'ELIG_CUS', '%_FIRST_SP', '%_TOTAL'])
    for k in ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15',
             '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30']:
        cus_elig_first_sp = cus_first_sp[cus_first_sp['CM2%'] >= float(k)]['cop_c_id'].count()
        elig_perc = cus_elig_first_sp / cus_first_sp['cop_c_id'].count() * 100
        elig_perc_from_total = cus_elig_first_sp / cus_amount * 100
        l = list([float(k), cus_elig_first_sp, elig_perc, elig_perc_from_total])
        df2 = pd.DataFrame([l], columns=df_first_sp.columns)
        df_first_sp = df_first_sp.append(df2, ignore_index=True)
    display(df_first_sp.round(2))
    print('_______________________________________')
    print('')
    df_renewals.round(2).to_excel(writer, sheet_name=site_dict[i], index=False, encoding='utf-8', startrow=1, startcol=0)
    df_first_sp.round(2).to_excel(writer, sheet_name=site_dict[i], index=False, encoding='utf-8', startrow=1, startcol=5)
    
writer.save()

SITE 1 [Deutschland]

Total amount of existing customers:  1241242
Renewals:  437426
First SP:  803816

Renewals Table


,THRESHOLD_%,ELIG_CUS,%_RENEW,%_TOTAL
0,1.0,423139,96.73,34.09
1,2.0,419456,95.89,33.79
2,3.0,414899,94.85,33.43
3,4.0,409191,93.55,32.97
4,5.0,402035,91.91,32.39
5,6.0,393194,89.89,31.68
6,7.0,382715,87.49,30.83
7,8.0,370191,84.63,29.82
8,9.0,355435,81.26,28.64
9,10.0,338548,77.40,27.27



First SP Table


,THRESHOLD_%,ELIG_CUS,%_FIRST_SP,%_TOTAL
0,1.0,769169,95.69,61.97
1,2.0,762740,94.89,61.45
2,3.0,754936,93.92,60.82
3,4.0,745555,92.75,60.07
4,5.0,734361,91.36,59.16
5,6.0,721054,89.70,58.09
6,7.0,705745,87.80,56.86
7,8.0,687917,85.58,55.42
8,9.0,667798,83.08,53.80
9,10.0,644830,80.22,51.95


_______________________________________

SITE 2 [International]

Total amount of existing customers:  4559
Renewals:  2127
First SP:  2432

Renewals Table


...

_______________________________________

SITE 32 [Greece]

Total amount of existing customers:  3472
Renewals:  579
First SP:  2893

Renewals Table


,THRESHOLD_%,ELIG_CUS,%_RENEW,%_TOTAL
0,1.0,207,35.75,5.96
1,2.0,192,33.16,5.53
2,3.0,177,30.57,5.10
3,4.0,167,28.84,4.81
4,5.0,152,26.25,4.38
5,6.0,140,24.18,4.03
6,7.0,132,22.80,3.80
7,8.0,123,21.24,3.54
8,9.0,104,17.96,3.00
9,10.0,92,15.89,2.65



First SP Table


,THRESHOLD_%,ELIG_CUS,%_FIRST_SP,%_TOTAL
0,1.0,922,31.87,26.56
1,2.0,841,29.07,24.22
2,3.0,764,26.41,22.00
3,4.0,715,24.71,20.59
4,5.0,642,22.19,18.49
5,6.0,586,20.26,16.88
6,7.0,515,17.80,14.83
7,8.0,452,15.62,13.02
8,9.0,412,14.24,11.87
9,10.0,364,12.58,10.48


_______________________________________



In [69]:
book = load_workbook(out_path)
writer = pd.ExcelWriter(out_path, engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

In [74]:
for i in np.sort(data['cop_pref_o_ext_s_id'].unique())[:25]:
    book[site_dict[i]].cell(row=1, column=1).value = "Renewals Table"
    book[site_dict[i]].cell(row=1, column=6).value = "First SP Table"
writer.save()

In [ ]:
# Calculations above don't correspond to control groups (2)

## Control Groups

Let's get from each customer his last eligibility status:

In [26]:
query = """
  SELECT *
        FROM(
            SELECT
            EC_C_ID,
            EC_EXT_CTI_ID,
            EC_BOX,
            EC_RULES,
            EC_TIMESTAMP,
            ROW_NUMBER() OVER ( PARTITION BY EC_C_ID ORDER BY EC_TIMESTAMP DESC ) LAST_ELIG
            FROM ELIGIBILITY_CHANGES
        ) q1
        WHERE LAST_ELIG = 1
"""

In [27]:
conn = psycopg2.connect("dbname='savings_plan_db' user='dba' host='savings-plan-prod-c-savings-plan-db-readonly.ccsod6gcmvcs.eu-central-1.rds.amazonaws.com' password='m4d_l0y_db4?'")
curs = conn.cursor()
data = pd.read_sql(query, con = conn)
conn.close()

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15871228 entries, 0 to 15871227
Data columns (total 6 columns):
ec_c_id          int64
ec_ext_cti_id    int64
ec_box           object
ec_rules         object
ec_timestamp     datetime64[ns]
last_elig        int64
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 726.5+ MB


In [29]:
data.head()

,ec_c_id,ec_ext_cti_id,ec_box,ec_rules,ec_timestamp,last_elig
0,1,0,Z,A|A|A,2018-07-31,1
1,6,0,Z,A|A|A,2018-07-31,1
2,10,0,Z,A|A|A,2018-07-31,1
3,12,0,Z,A|A|A,2018-07-31,1
4,14,0,Z,A|A|A,2018-07-31,1


In [36]:
data['ec_ext_cti_id'].unique()

array([   0,    6,    4,    5,   25,   24, 1005, 1004, 1000], dtype=int64)

In [37]:
data[data['ec_ext_cti_id'] != 0].sort_values(by='ec_c_id', ascending=False)

,ec_c_id,ec_ext_cti_id,ec_box,ec_rules,ec_timestamp,last_elig
15871225,20917236,6,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-08-07 09:33:04.092002,1
15871215,20917138,6,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-08-07 09:22:02.394286,1
15871172,20916944,6,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-08-07 08:55:20.495212,1
15871123,20916873,6,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-08-07 08:51:43.639807,1
15871031,20916761,6,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-08-07 08:38:12.516530,1
15871002,20916722,6,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-08-07 09:03:29.363444,1
15870977,20916691,6,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-08-07 08:48:59.604806,1
15870968,20916679,6,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-08-07 08:29:21.801476,1
15870829,20916503,6,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-08-07 08:01:06.829379,1
15870778,20916428,6,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-08-07 07:43:55.699754,1


And now, let's see customer_order_profile

In [30]:
query = """SELECT cop_c_id, cop_pref_o_ext_s_id, cop_g_order_count, cop_g_net_total, cop_g_cm2
FROM public.customer_order_profile
where cop_g_cm2 is not null
"""

In [31]:
conn = psycopg2.connect("dbname='savings_plan_db' user='dba' host='savings-plan-prod-c-savings-plan-db-readonly.ccsod6gcmvcs.eu-central-1.rds.amazonaws.com' password='m4d_l0y_db4?'")
curs = conn.cursor()
data_c = pd.read_sql(query, con = conn)
conn.close()

In [32]:
data_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9233523 entries, 0 to 9233522
Data columns (total 5 columns):
cop_c_id               int64
cop_pref_o_ext_s_id    int64
cop_g_order_count      int64
cop_g_net_total        float64
cop_g_cm2              float64
dtypes: float64(2), int64(3)
memory usage: 352.2 MB


In [33]:
data_c['CM2%'] = data_c['cop_g_cm2'] / data_c['cop_g_net_total'] * 100

In [34]:
data_c.sample(10)

,cop_c_id,cop_pref_o_ext_s_id,cop_g_order_count,cop_g_net_total,cop_g_cm2,CM2%
8475288,14565333,1,2,36.731092,7.203030,19.610171
7317935,12739031,18,1,33.048387,5.906387,17.871937
7765414,13505572,12,2,32.729508,2.921908,8.927442
6451037,10713243,26,2,118.183587,9.722387,8.226512
2862303,19358309,3,3,119.405511,-13.871054,-11.616762
9174577,15487398,12,1,36.778689,12.752686,34.674118
5852281,8644073,1,7,418.970247,57.873476,13.813266
7889954,13712737,21,7,213.385855,45.269045,21.214642
4118647,20731045,3,1,58.767370,3.104370,5.282472
1323702,17275642,4,3,382.216667,22.208669,5.810492


In [38]:
cus_data = data_c.merge(data, how='inner', left_on='cop_c_id', right_on='ec_c_id')

In [39]:
cus_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9233523 entries, 0 to 9233522
Data columns (total 12 columns):
cop_c_id               int64
cop_pref_o_ext_s_id    int64
cop_g_order_count      int64
cop_g_net_total        float64
cop_g_cm2              float64
CM2%                   float64
ec_c_id                int64
ec_ext_cti_id          int64
ec_box                 object
ec_rules               object
ec_timestamp           datetime64[ns]
last_elig              int64
dtypes: datetime64[ns](1), float64(3), int64(6), object(2)
memory usage: 915.8+ MB


In [40]:
cus_data.sample(5)

,cop_c_id,cop_pref_o_ext_s_id,cop_g_order_count,cop_g_net_total,cop_g_cm2,CM2%,ec_c_id,ec_ext_cti_id,ec_box,ec_rules,ec_timestamp,last_elig
6925230,12079910,16,2,62.431726,5.919134,9.480971,12079910,4,A,"A+,B-,C-,D+,E-,G-,H-,I+,L-,M-,N-,O+,P-|A-,B-,C...",2018-07-31 20:43:44.494025,1
8208484,14195084,4,1,25.816667,6.053424,23.447736,14195084,0,Z,A|A|A,2018-07-31 00:00:00.000000,1
8503934,14602702,7,6,394.371901,9.555551,2.422980,14602702,0,Z,A|A|A,2018-07-31 00:00:00.000000,1
84546,15678463,12,1,34.278689,8.213868,23.962025,15678463,6,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-07-31 21:30:57.278006,1
8769420,14951214,4,3,129.133333,33.229875,25.732996,14951214,5,A,"A+,B-,C-,D+,E-,G-,H-,I+,L-,M-,N-,O+,P+|A-,B-,C...",2018-07-31 21:18:58.353015,1


In [42]:
sp_eligible = cus_data[cus_data['ec_ext_cti_id'] != 0].sort_values(by='ec_c_id', ascending=False)

In [44]:
sp_eligible.sample(10)

,cop_c_id,cop_pref_o_ext_s_id,cop_g_order_count,cop_g_net_total,cop_g_cm2,CM2%,ec_c_id,ec_ext_cti_id,ec_box,ec_rules,ec_timestamp,last_elig
8464733,14551518,16,3,81.083583,22.893852,28.234880,14551518,5,B3,"A+,B-,C-,D+,E-,G-,H-,I+,L-,M-,N-,O+,P+|A-,B-,C...",2018-07-31 21:08:40.905820,1
9015891,15276630,1,1,48.200581,7.919581,16.430468,15276630,6,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-07-31 21:23:23.074515,1
6488088,10836674,4,2,80.466667,20.515630,25.495812,10836674,5,A,"A+,B-,C-,D+,E-,G-,H-,I+,L-,M-,N-,O+,P+|A-,B-,C...",2018-07-31 20:35:16.154565,1
609958,16360111,1,4,252.273332,51.907829,20.576027,16360111,4,A,"A+,B-,C-,D+,E-,G-,H-,I+,L-,M-,N-,O+,P-|A-,B-,C...",2018-07-31 21:42:16.372069,1
3681312,20510841,1,2,148.389900,18.066900,12.175290,20510841,4,A,"A+,B-,C-,D+,E-,G-,H-,I+,L-,M-,N-,O+,P-|A-,B-,C...",2018-08-03 21:59:29.695811,1
736710,16522974,12,8,643.672112,134.445031,20.887192,16522974,5,A,"A+,B-,C-,D+,E-,G-,H-,I+,L-,M-,N-,O+,P+|A-,B-,C...",2018-07-31 21:45:16.770597,1
5593467,7644160,7,2,175.462810,36.549643,20.830422,7644160,5,A,"A+,B-,C-,D+,E-,G-,H-,I+,L-,M-,N-,O+,P+|A-,B-,C...",2018-07-31 20:06:08.522422,1
7163151,12478591,7,7,355.024794,34.021544,9.582864,12478591,4,B4,"A+,B-,C-,D+,E-,G-,H-,I+,L-,M-,N-,O+,P-|A-,B-,C...",2018-07-31 20:51:08.094775,1
4920318,4442284,1,1,22.680672,8.847353,39.008337,4442284,6,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-07-31 19:34:10.747477,1
4672019,3028850,4,3,179.933367,16.633407,9.244204,3028850,4,A,"A+,B-,C-,D+,E-,G-,H-,I+,L-,M-,N-,O+,P-|A-,B-,C...",2018-07-31 19:22:34.531016,1


In [46]:
sp_eligible[sp_eligible['ec_timestamp'] >= '2018-07-01'].groupby(sp_eligible['ec_timestamp'].dt.date)['cop_c_id'].count()

ec_timestamp
2018-07-31    4012120
2018-08-01      13039
2018-08-02      17352
2018-08-03      13726
2018-08-04       5532
2018-08-05       7875
2018-08-06      13552
2018-08-07       6655
Name: cop_c_id, dtype: int64

Finally, let's locate those registered customer accounts in the last month

In [63]:
query = """SELECT c_id, c_registered_account, c_registration_ext_s_id, c_registration_date
FROM public.customers
where c_registered_account = True
and c_registration_date >= to_date('01/07/18', 'DD/MM/YY')
"""

In [64]:
conn = psycopg2.connect("dbname='savings_plan_db' user='dba' host='savings-plan-prod-c-savings-plan-db-readonly.ccsod6gcmvcs.eu-central-1.rds.amazonaws.com' password='m4d_l0y_db4?'")
curs = conn.cursor()
data_r = pd.read_sql(query, con = conn)
conn.close()

In [65]:
data_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259791 entries, 0 to 259790
Data columns (total 4 columns):
c_id                       259791 non-null int64
c_registered_account       259791 non-null bool
c_registration_ext_s_id    259791 non-null int64
c_registration_date        259791 non-null object
dtypes: bool(1), int64(2), object(1)
memory usage: 6.2+ MB


In [66]:
data_r.sample(5)

,c_id,c_registered_account,c_registration_ext_s_id,c_registration_date
41679,20859924,True,4,2018-07-31
152118,20640522,True,8,2018-07-04
127180,20683945,True,1,2018-07-09
207927,20667556,True,12,2018-07-08
81347,20822283,True,25,2018-07-26


In [67]:
data_r['c_registration_date'].min()

datetime.date(2018, 7, 1)

In [68]:
data_r['c_registered_account'].unique()

array([ True])

In [69]:
reg_cus_data = data_r.merge(cus_data, how='left', left_on='c_id', right_on='cop_c_id')

In [70]:
reg_cus_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259791 entries, 0 to 259790
Data columns (total 16 columns):
c_id                       259791 non-null int64
c_registered_account       259791 non-null bool
c_registration_ext_s_id    259791 non-null int64
c_registration_date        259791 non-null object
cop_c_id                   201621 non-null float64
cop_pref_o_ext_s_id        201621 non-null float64
cop_g_order_count          201621 non-null float64
cop_g_net_total            201621 non-null float64
cop_g_cm2                  201621 non-null float64
CM2%                       201621 non-null float64
ec_c_id                    201621 non-null float64
ec_ext_cti_id              201621 non-null float64
ec_box                     201621 non-null object
ec_rules                   201621 non-null object
ec_timestamp               201621 non-null datetime64[ns]
last_elig                  201621 non-null float64
dtypes: bool(1), datetime64[ns](1), float64(9), int64(2), object(3)
memory u

In [72]:
reg_cus_data.sample(10).round(2)

,c_id,c_registered_account,c_registration_ext_s_id,c_registration_date,cop_c_id,cop_pref_o_ext_s_id,cop_g_order_count,cop_g_net_total,cop_g_cm2,CM2%,ec_c_id,ec_ext_cti_id,ec_box,ec_rules,ec_timestamp,last_elig
105508,20897845,True,23,2018-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
94208,20618551,True,3,2018-07-02,20618551.0,3.0,1.0,27.93,-0.49,-1.76,20618551.0,0.0,Z,A|A|A,2018-07-31 00:00:00.000000,1.0
248987,20767736,True,21,2018-07-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
167921,20901166,True,1,2018-08-05,20901166.0,1.0,1.0,42.44,6.52,15.36,20901166.0,0.0,Z,"A+,B-,C-,D+,E-,G-,H-,I-,J-,K-|A+|A+",2018-08-05 16:02:04.696712,1.0
85668,20833066,True,4,2018-07-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
197002,20885596,True,1,2018-08-03,20885596.0,1.0,1.0,50.46,9.05,17.95,20885596.0,6.0,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-08-06 09:10:47.385894,1.0
63920,20822013,True,3,2018-07-26,20822013.0,3.0,1.0,29.00,-2.24,-7.72,20822013.0,0.0,Z,A|A|A,2018-07-31 00:00:00.000000,1.0
128959,20637632,True,12,2018-07-04,20637632.0,12.0,1.0,33.08,0.75,2.28,20637632.0,0.0,Z,A|A|A,2018-07-31 00:00:00.000000,1.0
222351,20845696,True,1,2018-07-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
30215,20914125,True,15,2018-08-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [86]:
zp_countries = np.sort(reg_cus_data['c_registration_ext_s_id'].unique())[:25]

In [92]:
zp_countries

array([ 1,  2,  3,  4,  7,  8, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 28, 29, 30, 31, 32], dtype=int64)

In [87]:
zp_reg_cus_data = reg_cus_data[reg_cus_data['c_registration_ext_s_id'].isin(zp_countries)]

In [83]:
import matplotlib.pyplot as plt
from IPython.display import display

In [160]:
for i in np.sort(reg_cus_data['c_registration_ext_s_id'].unique())[:25]:
    site_data = reg_cus_data[reg_cus_data['c_registration_ext_s_id'] == i]
    print('SITE ' + str(i) + ': ' + str(len(site_data)))

SITE 1: 47068
SITE 2: 418
SITE 3: 15353
SITE 4: 33693
SITE 7: 19711
SITE 8: 21647
SITE 11: 1249
SITE 12: 15027
SITE 14: 3774
SITE 15: 10428
SITE 16: 5633
SITE 18: 4317
SITE 19: 1840
SITE 20: 292
SITE 21: 6150
SITE 22: 4152
SITE 23: 1367
SITE 24: 2979
SITE 25: 5598
SITE 26: 9013
SITE 28: 2413
SITE 29: 858
SITE 30: 997
SITE 31: 1727
SITE 32: 1107


In [97]:
cols = list(['DATE']) + list(np.sort(zp_reg_cus_data['cop_pref_o_ext_s_id'].unique())[:25]) 

In [100]:
daily_reg_per_site = pd.DataFrame(columns = cols)

,DATE,1.0,2.0,3.0,4.0,7.0,8.0,11.0,12.0,14.0,...,22.0,23.0,24.0,25.0,26.0,28.0,29.0,30.0,31.0,32.0


In [101]:
daily_reg_per_site['DATE'] = np.sort(zp_reg_cus_data['c_registration_date'].unique())

In [106]:
daily_reg_per_site = daily_reg_per_site.set_index('DATE')

In [117]:
for i in np.sort(zp_reg_cus_data['c_registration_ext_s_id'].unique()):
    gr = zp_reg_cus_data[zp_reg_cus_data['cop_pref_o_ext_s_id'] == i].groupby('c_registration_date')['c_id'].nunique()
    for x, y in list(zip(gr.index, gr.values)):
        daily_reg_per_site.loc[x, i] = y

In [214]:
daily_reg_per_site = daily_reg_per_site.drop(daily_reg_per_site.index[len(daily_reg_per_site) - 1:])

In [248]:
daily_reg_per_site

,1.0,2.0,3.0,4.0,7.0,8.0,11.0,12.0,14.0,15.0,...,22.0,23.0,24.0,25.0,26.0,28.0,29.0,30.0,31.0,32.0
DATE,,,,,,,,,,,,,,,,,,,,,
2018-07-01,1396,12,389,892,540,540,36,339,92,247,...,89,40,42,164,193,51,18,8,42,20
2018-07-02,1355,5,387,964,580,726,49,527,88,345,...,130,45,68,171,230,70,29,16,44,18
2018-07-03,1260,5,352,918,562,636,28,457,111,337,...,138,33,70,138,200,46,33,22,56,20
2018-07-04,1137,3,336,860,525,649,29,460,114,315,...,122,39,56,124,194,64,27,19,43,24
2018-07-05,1062,5,326,813,495,532,35,397,101,268,...,130,29,48,133,164,34,22,19,51,22
2018-07-06,895,9,282,637,374,405,27,326,60,206,...,103,24,31,115,156,47,19,10,32,17
2018-07-07,824,7,240,647,370,303,20,275,86,185,...,69,28,39,99,132,45,15,16,38,15
2018-07-08,1155,6,320,818,505,549,24,336,100,243,...,123,38,44,164,182,41,20,11,35,12
2018-07-09,1311,7,344,982,662,798,38,474,98,317,...,152,29,70,141,245,66,21,21,46,35


In [215]:
daily_reg_per_site.fillna(0).apply(sum, axis=1)

DATE
2018-07-01    5610
2018-07-02    6342
2018-07-03    5896
2018-07-04    5542
2018-07-05    5048
2018-07-06    4128
2018-07-07    3742
2018-07-08    5204
2018-07-09    6407
2018-07-10    5955
2018-07-11    5775
2018-07-12    5017
2018-07-13    4169
2018-07-14    3626
2018-07-15    4811
2018-07-16    5932
2018-07-17    5761
2018-07-18    5217
2018-07-19    4776
2018-07-20    4072
2018-07-21    3661
2018-07-22    5043
2018-07-23    5930
2018-07-24    5486
2018-07-25    5095
2018-07-26    4479
2018-07-27    1006
2018-07-28     434
2018-07-29     821
2018-07-30    1804
2018-07-31    5702
2018-08-01    5850
2018-08-02    5142
2018-08-03    4274
2018-08-04    3589
2018-08-05    4196
2018-08-06    3022
dtype: int64

In [216]:
reg_rel_values_per_site = daily_reg_per_site.apply(lambda x: x/x.sum() * 100, axis=1).round(2)

In [217]:
daily_reg_per_site.apply(lambda x: x/x.sum() * 100, axis=1).round(2)[daily_reg_per_site.columns[:12]]

,1.0,2.0,3.0,4.0,7.0,8.0,11.0,12.0,14.0,15.0,16.0,18.0
DATE,,,,,,,,,,,,
2018-07-01,24.88,0.21,6.93,15.90,9.63,9.63,0.64,6.04,1.64,4.40,2.60,1.89
2018-07-02,21.37,0.08,6.10,15.20,9.15,11.45,0.77,8.31,1.39,5.44,2.33,1.92
2018-07-03,21.37,0.08,5.97,15.57,9.53,10.79,0.47,7.75,1.88,5.72,1.95,2.14
2018-07-04,20.52,0.05,6.06,15.52,9.47,11.71,0.52,8.30,2.06,5.68,1.79,1.88
2018-07-05,21.04,0.10,6.46,16.11,9.81,10.54,0.69,7.86,2.00,5.31,1.66,1.60
2018-07-06,21.68,0.22,6.83,15.43,9.06,9.81,0.65,7.90,1.45,4.99,1.79,2.52
2018-07-07,22.02,0.19,6.41,17.29,9.89,8.10,0.53,7.35,2.30,4.94,2.19,1.87
2018-07-08,22.19,0.12,6.15,15.72,9.70,10.55,0.46,6.46,1.92,4.67,3.19,1.98
2018-07-09,20.46,0.11,5.37,15.33,10.33,12.46,0.59,7.40,1.53,4.95,3.48,1.51


In [218]:
daily_reg_per_site.apply(lambda x: x/x.sum() * 100, axis=1).round(2)[daily_reg_per_site.columns[12:]]

,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,28.0,29.0,30.0,31.0,32.0
DATE,,,,,,,,,,,,,
2018-07-01,0.59,0.05,3.07,1.59,0.71,0.75,2.92,3.44,0.91,0.32,0.14,0.75,0.36
2018-07-02,0.60,0.08,2.87,2.05,0.71,1.07,2.70,3.63,1.10,0.46,0.25,0.69,0.28
2018-07-03,0.71,0.12,3.12,2.34,0.56,1.19,2.34,3.39,0.78,0.56,0.37,0.95,0.34
2018-07-04,0.65,0.13,2.81,2.20,0.70,1.01,2.24,3.50,1.15,0.49,0.34,0.78,0.43
2018-07-05,0.61,0.34,2.95,2.58,0.57,0.95,2.63,3.25,0.67,0.44,0.38,1.01,0.44
2018-07-06,0.97,0.29,2.98,2.50,0.58,0.75,2.79,3.78,1.14,0.46,0.24,0.78,0.41
2018-07-07,0.75,0.08,2.83,1.84,0.75,1.04,2.65,3.53,1.20,0.40,0.43,1.02,0.40
2018-07-08,0.94,0.15,2.92,2.36,0.73,0.85,3.15,3.50,0.79,0.38,0.21,0.67,0.23
2018-07-09,0.83,0.09,2.67,2.37,0.45,1.09,2.20,3.82,1.03,0.33,0.33,0.72,0.55


In [247]:
#Median values when looking at how are registered customers distributed per site
reg_rel_values_per_site.describe().loc['50%'].round(2)

1.0     21.37
2.0      0.12
3.0      6.93
4.0     15.24
7.0      9.63
8.0     10.87
11.0     0.61
12.0     7.14
14.0     1.65
15.0     4.74
16.0     2.95
18.0     1.92
19.0     0.82
20.0     0.13
21.0     2.95
22.0     1.96
23.0     0.70
24.0     1.04
25.0     2.62
26.0     3.53
28.0     1.03
29.0     0.44
30.0     0.32
31.0     0.78
32.0     0.40
Name: 50%, dtype: float64

In [220]:
zp_reg_cus_data.head(10)

,c_id,c_registered_account,c_registration_ext_s_id,c_registration_date,cop_c_id,cop_pref_o_ext_s_id,cop_g_order_count,cop_g_net_total,cop_g_cm2,CM2%,ec_c_id,ec_ext_cti_id,ec_box,ec_rules,ec_timestamp,last_elig
1,20687498,True,8,2018-07-10,20687498.0,8.0,2.0,32.837283,5.535283,16.856703,20687498.0,5.0,A,"A+,B-,C-,D+,E-,G-,H-,I+,L-,M-,N-,O+,P+|A-,B-,C...",2018-07-31 22:35:14.730085,1.0
2,20718112,True,3,2018-07-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
6,20767855,True,22,2018-07-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
7,20768144,True,15,2018-07-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
8,20785910,True,1,2018-07-22,20785910.0,1.0,1.0,27.904100,7.770900,27.848596,20785910.0,6.0,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-07-31 22:30:56.744814,1.0
9,20785857,True,4,2018-07-22,20785857.0,4.0,1.0,48.591700,-1.841700,-3.790153,20785857.0,0.0,Z,A|A|A,2018-07-31 00:00:00.000000,1.0
10,20785978,True,8,2018-07-22,20785978.0,8.0,1.0,53.498857,2.970357,5.552188,20785978.0,6.0,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-07-31 22:31:17.756786,1.0
11,20786040,True,4,2018-07-22,20786040.0,4.0,1.0,49.075000,1.706400,3.477127,20786040.0,6.0,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-07-31 22:31:36.575233,1.0
12,20786134,True,1,2018-07-22,20786134.0,1.0,1.0,17.327100,5.081300,29.325738,20786134.0,6.0,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-07-31 22:32:23.467107,1.0
13,20786133,True,14,2018-07-22,20786133.0,14.0,1.0,33.752100,-0.187900,-0.556706,20786133.0,6.0,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-07-31 22:34:16.884711,1.0


In [221]:
zp_reg_cus_data.columns

Index(['c_id', 'c_registered_account', 'c_registration_ext_s_id',
       'c_registration_date', 'cop_c_id', 'cop_pref_o_ext_s_id',
       'cop_g_order_count', 'cop_g_net_total', 'cop_g_cm2', 'CM2%', 'ec_c_id',
       'ec_ext_cti_id', 'ec_box', 'ec_rules', 'ec_timestamp', 'last_elig'],
      dtype='object')

In [222]:
zp_reg_cus_data['ec_ext_cti_id'].unique()

array([   5.,   nan,    6.,    0.,    4., 1000.,   24.,   25., 1004.,
       1005.])

In [223]:
len(zp_reg_cus_data)

216811

In [224]:
#len 168601 with eligibility status
zp_reg_cus_data_not_null = zp_reg_cus_data[~zp_reg_cus_data.apply(lambda x: x.isnull().any(), axis=1)]

In [225]:
len(zp_reg_cus_data_not_null)

168601

In [226]:
zp_reg_cus_sp_elig = zp_reg_cus_data_not_null[zp_reg_cus_data_not_null['ec_ext_cti_id'] != 0]

In [227]:
zp_reg_cus_sp_elig['ec_ext_cti_id'].value_counts()

6.0       67657
5.0        5573
4.0        5479
24.0        208
1000.0      196
25.0         74
1004.0       63
1005.0       10
Name: ec_ext_cti_id, dtype: int64

In [228]:
zp_reg_cus_data.head()

,c_id,c_registered_account,c_registration_ext_s_id,c_registration_date,cop_c_id,cop_pref_o_ext_s_id,cop_g_order_count,cop_g_net_total,cop_g_cm2,CM2%,ec_c_id,ec_ext_cti_id,ec_box,ec_rules,ec_timestamp,last_elig
1,20687498,True,8,2018-07-10,20687498.0,8.0,2.0,32.837283,5.535283,16.856703,20687498.0,5.0,A,"A+,B-,C-,D+,E-,G-,H-,I+,L-,M-,N-,O+,P+|A-,B-,C...",2018-07-31 22:35:14.730085,1.0
2,20718112,True,3,2018-07-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
6,20767855,True,22,2018-07-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
7,20768144,True,15,2018-07-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
8,20785910,True,1,2018-07-22,20785910.0,1.0,1.0,27.904100,7.770900,27.848596,20785910.0,6.0,N,"A+,B-,C-,D+,E-,G-,H-,I-,J+|A-,B-,C-,D+,E+|A-,B...",2018-07-31 22:30:56.744814,1.0


In [229]:
sp_tiers = [4, 5, 6, 24, 25, 1000, 1004, 1005]

In [237]:
daily_elig_per_site = pd.DataFrame(columns = cols)

In [238]:
daily_elig_per_site['DATE'] = np.sort(zp_reg_cus_data['c_registration_date'].unique())

In [239]:
daily_elig_per_site = daily_elig_per_site.set_index('DATE')

In [240]:
for i in np.sort(zp_reg_cus_data['c_registration_ext_s_id'].unique()):
    site_data = zp_reg_cus_data[zp_reg_cus_data['c_registration_ext_s_id'] == i]
    elig_cus_per_site = site_data[site_data['ec_ext_cti_id'].isin(sp_tiers)]
    perc_elig = elig_cus_per_site.groupby('c_registration_date')['c_id'].nunique() /\
    site_data.groupby('c_registration_date')['c_id'].nunique() * 100
    for x, y in list(zip(perc_elig.index, perc_elig.values)):
        daily_elig_per_site.loc[x, i] = float(y)

In [241]:
daily_elig_per_site = daily_elig_per_site.drop(daily_elig_per_site.index[len(daily_elig_per_site) - 1:])

In [242]:
daily_elig_per_site.fillna(0).round(2)

,1.0,2.0,3.0,4.0,7.0,8.0,11.0,12.0,14.0,15.0,...,22.0,23.0,24.0,25.0,26.0,28.0,29.0,30.0,31.0,32.0
DATE,,,,,,,,,,,,,,,,,,,,,
2018-07-01,64.92,35.00,24.00,34.53,40.11,36.93,38.46,36.51,51.00,38.06,...,17.53,38.10,29.03,77.84,20.76,43.55,42.11,31.25,31.25,47.83
2018-07-02,65.69,55.56,23.06,37.66,43.08,39.37,46.15,38.95,37.96,45.58,...,17.86,39.22,27.27,81.38,16.96,45.78,41.94,26.67,34.00,44.00
2018-07-03,63.96,27.27,23.92,40.27,43.78,38.91,48.28,38.31,45.08,36.66,...,20.13,52.78,25.00,74.03,15.21,36.67,21.62,17.65,26.67,35.71
2018-07-04,65.41,0.00,16.84,34.93,40.60,36.83,43.33,35.88,40.88,41.38,...,26.62,43.90,24.14,73.29,21.22,44.00,25.93,18.18,30.61,35.14
2018-07-05,66.72,44.44,20.45,36.51,42.40,35.48,43.59,34.92,45.54,34.55,...,13.61,40.62,20.00,79.47,17.89,30.00,41.67,9.68,44.07,48.39
2018-07-06,61.38,31.25,17.67,36.23,39.56,37.62,45.16,34.08,48.53,39.48,...,21.10,46.15,8.00,68.18,17.95,40.32,45.45,9.09,27.27,50.00
2018-07-07,67.34,60.00,17.28,35.97,45.14,39.06,59.09,33.33,35.92,37.44,...,14.81,48.28,21.67,69.57,12.64,50.00,35.29,17.39,28.26,47.83
2018-07-08,65.41,25.00,18.52,37.68,42.81,42.36,42.31,38.78,40.74,38.06,...,22.05,35.90,18.18,75.28,19.75,39.58,23.81,22.22,31.71,30.43
2018-07-09,64.21,44.44,21.22,36.84,45.26,39.29,48.72,40.65,51.28,38.31,...,19.64,22.58,21.10,80.77,19.43,57.69,23.81,9.68,49.06,62.50


### Registered customers per day

In [249]:
daily_reg_per_site.fillna(0).apply(sum, axis=1)

DATE
2018-07-01    5610
2018-07-02    6342
2018-07-03    5896
2018-07-04    5542
2018-07-05    5048
2018-07-06    4128
2018-07-07    3742
2018-07-08    5204
2018-07-09    6407
2018-07-10    5955
2018-07-11    5775
2018-07-12    5017
2018-07-13    4169
2018-07-14    3626
2018-07-15    4811
2018-07-16    5932
2018-07-17    5761
2018-07-18    5217
2018-07-19    4776
2018-07-20    4072
2018-07-21    3661
2018-07-22    5043
2018-07-23    5930
2018-07-24    5486
2018-07-25    5095
2018-07-26    4479
2018-07-27    1006
2018-07-28     434
2018-07-29     821
2018-07-30    1804
2018-07-31    5702
2018-08-01    5850
2018-08-02    5142
2018-08-03    4274
2018-08-04    3589
2018-08-05    4196
2018-08-06    3022
dtype: int64

### Median values when looking at how are registered customers distributed per site

In [250]:
#Index values are Shop IDs
reg_rel_values_per_site.describe().loc['50%'].round(2)

1.0     21.37
2.0      0.12
3.0      6.93
4.0     15.24
7.0      9.63
8.0     10.87
11.0     0.61
12.0     7.14
14.0     1.65
15.0     4.74
16.0     2.95
18.0     1.92
19.0     0.82
20.0     0.13
21.0     2.95
22.0     1.96
23.0     0.70
24.0     1.04
25.0     2.62
26.0     3.53
28.0     1.03
29.0     0.44
30.0     0.32
31.0     0.78
32.0     0.40
Name: 50%, dtype: float64

### Median values per site in terms of eligible customers

In [244]:
#Index values are Shop IDs
daily_elig_per_site.fillna(0).round(2).describe().loc['50%'].round(2)

1.0     64.65
2.0     36.84
3.0     17.90
4.0     35.21
7.0     40.60
8.0     36.29
11.0    45.00
12.0    34.92
14.0    39.53
15.0    36.84
16.0    40.00
18.0    32.00
19.0    16.22
20.0    60.00
21.0    25.37
22.0    17.14
23.0    34.69
24.0    16.51
25.0    71.72
26.0    16.25
28.0    43.68
29.0    27.27
30.0    16.00
31.0    31.25
32.0    45.71
Name: 50%, dtype: float64

In [284]:
df = pd.DataFrame(columns=cols[1:])

In [285]:
for i in daily_reg_per_site.fillna(0).apply(sum, axis=1).values:
    reg_cus = reg_rel_values_per_site.describe().loc['50%'].round(2).values / 100 * i
    elig_cus = list(reg_cus * daily_elig_per_site.fillna(0).round(2).describe().loc['50%'].round(2).values / 100)
    row = pd.DataFrame([elig_cus], columns=df.columns)
    df = pd.concat([df, row], ignore_index=True)

In [293]:
df = df.set_index([daily_reg_per_site.fillna(0).apply(sum, axis=1).index])

In [295]:
df.loc['SUM_ELIG_CUS'] = df.apply(sum, axis=0)

### Approximation of the number of customers being eligible for any SP per day and site (column: shop_id, index: date)

In [297]:
df.round(2)

,1.0,2.0,3.0,4.0,7.0,8.0,11.0,12.0,14.0,15.0,...,22.0,23.0,24.0,25.0,26.0,28.0,29.0,30.0,31.0,32.0
DATE,,,,,,,,,,,,,,,,,,,,,
2018-07-01,775.06,2.48,69.59,301.03,219.34,221.30,15.40,139.87,36.59,97.96,...,18.85,13.62,9.63,105.42,32.18,25.24,6.73,2.87,13.67,10.26
2018-07-02,876.19,2.80,78.67,340.31,247.96,250.17,17.41,158.12,41.37,110.75,...,21.31,15.40,10.89,119.17,36.38,28.53,7.61,3.25,15.46,11.60
2018-07-03,814.57,2.61,73.14,316.38,230.52,232.58,16.18,147.00,38.46,102.96,...,19.81,14.32,10.12,110.79,33.82,26.53,7.07,3.02,14.37,10.78
2018-07-04,765.67,2.45,68.75,297.38,216.68,218.62,15.21,138.18,36.15,96.78,...,18.62,13.46,9.52,104.14,31.79,24.93,6.65,2.84,13.51,10.13
2018-07-05,697.42,2.23,62.62,270.88,197.37,199.13,13.86,125.86,32.93,88.15,...,16.96,12.26,8.67,94.86,28.96,22.71,6.06,2.58,12.30,9.23
2018-07-06,570.31,1.82,51.21,221.51,161.40,162.84,11.33,102.92,26.92,72.08,...,13.87,10.02,7.09,77.57,23.68,18.57,4.95,2.11,10.06,7.55
2018-07-07,516.98,1.65,46.42,200.80,146.30,147.61,10.27,93.30,24.41,65.34,...,12.57,9.09,6.43,70.31,21.47,16.84,4.49,1.92,9.12,6.84
2018-07-08,718.97,2.30,64.55,279.25,203.46,205.28,14.28,129.75,33.94,90.87,...,17.48,12.64,8.94,97.79,29.85,23.41,6.24,2.66,12.68,9.51
2018-07-09,885.17,2.83,79.48,343.80,250.50,252.74,17.59,159.74,41.79,111.88,...,21.52,15.56,11.00,120.39,36.75,28.83,7.69,3.28,15.62,11.71


In [299]:
df.round(2).to_csv('approx_elig_customers_per_shop_and_day.csv', sep=';', encoding = 'utf-8')

Last row from the table above shows an approximation to the number of eligible customers per shop after 37 days in order to define the matching patterns enabling some customer IDs to access the control group.

In [303]:
#TOP 7 shops
df[[1, 3, 4, 7, 8, 12, 15]].loc['SUM_ELIG_CUS'] * 0.01

1.0     232.883050
3.0      20.909859
4.0      90.451510
7.0      65.904816
8.0      66.493829
12.0     42.027860
15.0     29.434915
Name: SUM_ELIG_CUS, dtype: float64

Figures above try to approximate the number of customers in Top 7 shops belonging to the control groups we would have after 37 days, having supossed that the matching pattern allows 1% of the eligible customers take part of the control group.

Another option could be just enable more than one matching patterns (customer IDs ending in '03', '06', '09' for example) and then stop including customers in it once a suitable amount has been reached.